In [1]:
import os
os.chdir('..')

import pandas as pd

from scripts.crash_data_analysis import CrashDataAnalysis
cda = CrashDataAnalysis()

In [2]:
# df = pd.read_csv('data/crash_data_raw.csv', low_memory=False)

# df.groupby('LIGHT_CONDITION').size()

# df['day_or_night'] = df.LIGHT_CONDITION.map({
#     '  ': 'Unknown'
#     , 'DARK-LIGHTED': 'Night'
#     , 'DARK-UNLIGHTED': 'Night'
#     , 'DAWN OR DUSK': 'Night'
#     , 'DAY LIGHT': 'Day'
#     , 'Dark-Lighted': 'Night'
#     , 'Dark-Unlighted': 'Night'
#     , 'Dawn or Dusk': 'Night'
#     , 'Daylight': 'Day'
#     , 'UNDER INVESTIGATION': 'Unknown'
# })

# df.groupby(['day_or_night', 'LIGHT_CONDITION']).size()

In [3]:
# Aggregate fatality crashes by year and day of year

query = """
select
to_char(reported_date, 'YYYY-MM') as year_month
, day_or_night
, count(*) as num_crashes
, sum(fatality::int) as deadly_crashes
, sum(case when pedestrian_ind > 0 then 1 else 0 end) as pedestrian_involved_crash
, sum(case when bicycle_ind > 0 then 1 else 0 end) as bicycle_involved_crash
, sum(case when fatality and pedestrian_ind > 0 then 1 else 0 end) as pedestrian_involved_fatality
, sum(case when fatality and bicycle_ind > 0 then 1 else 0 end) as bicycle_involved_fatality

from crashes

group by 1,2
order by 1,2
"""

crashes_by_year_month_long = pd.read_sql(query, cda.conn)

In [4]:
crashes_by_year_month_long

,year_month,day_or_night,num_crashes,deadly_crashes,pedestrian_involved_crash,bicycle_involved_crash,pedestrian_involved_fatality,bicycle_involved_fatality
0,2013-01,Day,1048,2,28,12,0,0
1,2013-01,Night,747,2,36,4,1,0
2,2013-01,Unknown,49,0,0,0,0,0
3,2013-02,Day,987,0,27,8,0,0
4,2013-02,Night,537,0,15,3,0,0
...,...,...,...,...,...,...,...,...
419,2023-11,None,8,0,0,0,0,0
420,2023-12,Day,210,0,4,2,0,0
421,2023-12,Night,152,0,2,2,0,0
422,2023-12,Unknown,116,0,5,1,0,0
